In [1]:
import pandas as pd
import numpy as np
import sklearn
import os
import math

In [17]:
def make_data(directory):
    files = os.listdir(directory)
    d = {}
    for file in files:
        file_name = os.path.basename(file)
        os.chdir(directory)
        f = open(file_name)
        file_content = f.readlines()[:-1]
        for i in file_content:
            l = i.strip().split()
            if len(l)>2:
                k = ' '.join(l[:-1])
                v = l[-1]
            else:
                k = l[0]
                v = l[-1]
#             else:
#                 print(i,"n")
#                 k,v = i.strip().split()
#                 print(k,v)
            if k in d:
                d[k].append(v)
            else:
                d[k]=[v]
                
    return d        
m = make_data(r"C:\Users\admin\Downloads\dataset\train")
n = make_data(r"C:\Users\admin\Downloads\dataset\test")  

In [2]:
train_data = pd.DataFrame(m)
test_data = pd.DataFrame(n)
train_data.to_csv(r"C:\Users\admin\Downloads\dataset\train.csv",index=False)
test_data.to_csv(r"C:\Users\admin\Downloads\dataset\test.csv",index=False)

NameError: name 'm' is not defined

In [188]:
train_data = pd.read_csv(r"C:\Users\admin\Downloads\dataset\train.csv")
test_data = pd.read_csv(r"C:\Users\admin\Downloads\dataset\test.csv")

In [189]:
id_ = test_data['ID']

In [190]:
ids = list(train_data['Delivery_person_ID'])
te_ids = list(test_data['Delivery_person_ID'])
count=0
for id in te_ids:
    if id in ids:
        count+=1

In [191]:
test_data.shape,count

((11399, 19), 11399)

In [192]:
for col in train_data.columns:
    print(col,len(train_data[col].unique()))

ID 45592
Delivery_person_ID 1320
Delivery_person_Age 23
Delivery_person_Ratings 29
Restaurant_latitude 657
Restaurant_longitude 518
Delivery_location_latitude 4373
Delivery_location_longitude 4373
Order_Date 44
Time_Orderd 177
Time_Order_picked 193
Weather conditions 7
Road_traffic_density 5
Vehicle_condition 4
Type_of_order 4
Type_of_vehicle 4
multiple_deliveries 5
Festival 3
City 4
Time_taken (min) 45


In [193]:
def to_numeric(df,is_train=None):
    bins= [18,25,30,40,50,100]
    df = df.replace('NaN',None)
    numeric_columns = ['Delivery_person_Age',
           'Delivery_person_Ratings', 'Restaurant_latitude',
           'Restaurant_longitude', 'Delivery_location_latitude',
           'Delivery_location_longitude',
        'Vehicle_condition','multiple_deliveries']
    for col in numeric_columns:
        df[col] = df[col].astype(np.number)
    df['Order_Date'] = pd.to_datetime(df['Order_Date'])
    df['Order_month'] = df['Order_Date'].dt.month
    df['Order_day'] = df['Order_Date'].dt.day
    df['Order_weekday'] = df['Order_Date'].dt.weekday
    df["Is_weekend"] = np.where((df["Order_weekday"] >4),1,0)
    df["Is_mon"] = np.where(df["Order_weekday"] ==1,1,0)
    temp = df['Time_Orderd'].str.split(':', expand=True)
    df['Order_hrs'],df['Order_mins'] = temp[0],temp[1]
    temp=df['Time_Order_picked'].str.split(':', expand=True)
    df['Order_p_hrs'],df['Order_p_mins'] = temp[0],temp[1]
    df['city'] = df['Delivery_person_ID'].str[:4]   
    dcols = ['Order_hrs','Order_mins','Order_p_hrs','Order_p_mins']
    df['distance'] = (df['Restaurant_latitude']-df['Delivery_location_latitude'])**2 + (df['Restaurant_longitude']-df['Delivery_location_longitude'])**2
    if is_train:
#         df['distance'] = (df['Restaurant_latitude']-df['Delivery_location_latitude'])**2 + (df['Restaurant_longitude']-df['Delivery_location_longitude'])**2
        df['time/distance'] = df["Time_taken (min)"].astype(np.number)/df['distance']
        df_temp = df.groupby('Delivery_person_ID')["time/distance"].mean()
        df = df.merge(df_temp, on='Delivery_person_ID',how='left')
        df = df.dropna()
        df = df.drop('time/distance_x',axis=1,)

    for col in dcols:
        df[col] = df[col].replace('NaN',0)
        df = df.fillna(0)
        df[col] = df[col].apply(int)
    labels = ['NAdult','YAdult','OldAdult','VoldAdult','VVoldAdult']
    df['AgeGroup'] = pd.cut(df['Delivery_person_Age'], bins=bins, labels=labels, right=False)
    cat_cols = ["Weather conditions", "Road_traffic_density", "Type_of_order", "Type_of_vehicle", "Festival","city","AgeGroup"]
    for col in cat_cols:
        df[col] = df[col].astype('category')
    
    

    df = df.drop(["Time_Orderd",'Time_Order_picked','ID','Order_Date','City'],axis=1,)

    return df
train_data = to_numeric(train_data,is_train=True)
test_data = to_numeric(test_data)

In [194]:
test_data.shape

(11399, 26)

In [195]:
test_data = test_data.merge(train_data[['Delivery_person_ID','time/distance_y']].drop_duplicates('Delivery_person_ID',keep='last'),on='Delivery_person_ID',how='left')

In [196]:
target_y = train_data['Time_taken (min)']
train_data.drop(['Time_taken (min)','Delivery_person_ID'],axis=1,inplace=True)
test_data.drop('Delivery_person_ID',axis=1,inplace=True)

In [197]:
train_data.shape,test_data.shape

((41367, 26), (11399, 26))

In [198]:
all_data = pd.concat([train_data,test_data])
all_data.shape

(52766, 26)

In [199]:
numeric_columns = ['Delivery_person_Age',
           'Delivery_person_Ratings', 'Restaurant_latitude',
           'Restaurant_longitude', 'Delivery_location_latitude',
           'Delivery_location_longitude',
        'Vehicle_condition','multiple_deliveries','time/distance_y','distance']


In [200]:
from scipy.stats import norm,skew
skewed_feats = all_data[numeric_columns].apply(lambda x:skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({"Skew":skewed_feats})

In [201]:
skewed_feats

distance                       32.616312
time/distance_y                 0.722909
multiple_deliveries             0.329884
Vehicle_condition               0.019327
Delivery_location_latitude     -0.702351
Delivery_person_Age            -0.801478
Restaurant_latitude            -1.144903
Delivery_location_longitude    -2.973118
Restaurant_longitude           -3.060907
Delivery_person_Ratings        -5.763150
dtype: float64

In [204]:
skewness = skewness[abs(skewness)>0.75]
from scipy.special import boxcox1p
skewed_features = skewness.index
lam =0.15
for feat in skewed_features:
    all_data[feat] = boxcox1p(all_data[feat],lam)
cat_cols = ["Weather conditions", "Road_traffic_density", "Type_of_order", "Type_of_vehicle", "Festival","city","AgeGroup"]
for col in cat_cols:
    all_data[col] = all_data[col].astype('category') 
all_data = pd.get_dummies(all_data)
print(all_data.shape)
train_data = all_data.iloc[:41367]
test_data = all_data.iloc[41367:]

(52766, 69)


In [205]:
train_data.columns

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Vehicle_condition',
       'multiple_deliveries', 'Order_month', 'Order_day', 'Order_weekday',
       'Is_weekend', 'Is_mon', 'Order_hrs', 'Order_mins', 'Order_p_hrs',
       'Order_p_mins', 'distance', 'time/distance_y', 'Weather conditions_0',
       'Weather conditions_Cloudy', 'Weather conditions_Fog',
       'Weather conditions_Sandstorms', 'Weather conditions_Stormy',
       'Weather conditions_Sunny', 'Weather conditions_Windy',
       'Road_traffic_density_0', 'Road_traffic_density_High',
       'Road_traffic_density_Jam', 'Road_traffic_density_Low',
       'Road_traffic_density_Medium', 'Type_of_order_Buffet',
       'Type_of_order_Drinks', 'Type_of_order_Meal', 'Type_of_order_Snack',
       'Type_of_vehicle_bicycle', 'Type_of_vehicle_electric_scooter',
       'Type_of_vehicle_motorcycle', 'Type_of_v

In [173]:
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import 
# poly=PolynomialFeatures(3)
# num_train_data = poly.fit_transform(train_data[numeric_columns])
# num_test_data = poly.transform(test_data[numeric_columns])

SyntaxError: invalid syntax (<ipython-input-173-a2303ce4ebe8>, line 3)

In [174]:
num_train_data.shape

(41367, 220)

In [207]:
unnum_cols = list(set(train_data.columns) - set(numeric_columns))

In [208]:
pd.DataFrame(num_train_data).shape,train_data[unnum_cols].shape

((41367, 220), (41367, 59))

In [209]:
t_data = np.concatenate([num_train_data,train_data[unnum_cols].to_numpy()],axis=1)
te_data = np.concatenate([num_test_data,test_data[unnum_cols].to_numpy()],axis=1)


In [210]:
num_test_data.shape

(11399, 220)

In [211]:
from sklearn.model_selection import train_test_split
tr,te,ty,tey = train_test_split(t_data, target_y, test_size=0.30, random_state=42) 

In [212]:
pred.shape

(11399,)

In [213]:
te_data.shape

(11399, 279)

In [221]:
!pip install lightgbm

In [234]:
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV
lgb=lgbm.LGBMRegressor(objective='regression')
params_grid = {'n_estimators':range(200, 800, 100), 'num_leaves':range(5,60,10),'learning_rate':[0.05]}
clf = GridSearchCV(lgb, params_grid, cv=2)
clf.fit(tr, ty)

GridSearchCV(cv=2, estimator=LGBMRegressor(objective='regression'),
             param_grid={'learning_rate': [0.05],
                         'n_estimators': range(200, 800, 100),
                         'num_leaves': range(5, 60, 10)})

In [235]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
predv = clf.predict(te)
rmse = (np.sqrt(mean_squared_error(tey, predv)))
r2 = r2_score(tey, predv)
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))
predt = clf.predict(te_data)

RMSE: 4.18
R2: 0.80


In [237]:
clf.best_params_

{'learning_rate': 0.05, 'n_estimators': 300, 'num_leaves': 55}

In [240]:
#cat
params: {'depth': 5,
  'l2_leaf_reg': 1,
  'iterations': 500,
  'learning_rate': 0.05}
         
lgb_params = {'learning_rate': 0.05, 'n_estimators': 300, 'num_leaves': 55}

In [245]:
import lightgbm as lgbm
lgb_wh=lgbm.LGBMRegressor(objective='regression',learning_rate= 0.05, n_estimators=300, num_leaves=55)
lgb_wh.fit(t_data, target_y)

LGBMRegressor(learning_rate=0.05, n_estimators=300, num_leaves=55,
              objective='regression')

In [246]:
predt = lgb_wh.predict(te_data)

In [248]:
import catboost as cb
model = cb.CatBoostRegressor(loss_function='RMSE',iterations=300,depth=4,learning_rate=0.05)
grid = {'iterations': [500,600],
         'learning_rate': [0.05],
         'depth': [5,8,10],
         'l2_leaf_reg': [ 0.5, 1,2]}
model.grid_search(grid,t_data, target_y)

0:	learn: 26.8269277	test: 26.9913530	best: 26.9913530 (0)	total: 58.3ms	remaining: 29.1s
1:	learn: 25.5891480	test: 25.7543130	best: 25.7543130 (1)	total: 137ms	remaining: 34.1s
2:	learn: 24.4434412	test: 24.6090300	best: 24.6090300 (2)	total: 200ms	remaining: 33.1s
3:	learn: 23.3267109	test: 23.4880225	best: 23.4880225 (3)	total: 275ms	remaining: 34.1s
4:	learn: 22.2821048	test: 22.4397778	best: 22.4397778 (4)	total: 335ms	remaining: 33.2s
5:	learn: 21.3093423	test: 21.4645669	best: 21.4645669 (5)	total: 394ms	remaining: 32.4s
6:	learn: 20.3628510	test: 20.5181635	best: 20.5181635 (6)	total: 448ms	remaining: 31.5s
7:	learn: 19.4575486	test: 19.6108724	best: 19.6108724 (7)	total: 511ms	remaining: 31.4s
8:	learn: 18.6226484	test: 18.7735882	best: 18.7735882 (8)	total: 569ms	remaining: 31.1s
9:	learn: 17.8292509	test: 17.9778147	best: 17.9778147 (9)	total: 625ms	remaining: 30.6s
10:	learn: 17.0578789	test: 17.2063649	best: 17.2063649 (10)	total: 678ms	remaining: 30.2s
11:	learn: 16.3237

94:	learn: 5.0384200	test: 5.0373873	best: 5.0373873 (94)	total: 6.4s	remaining: 27.3s
95:	learn: 5.0343884	test: 5.0330901	best: 5.0330901 (95)	total: 6.48s	remaining: 27.3s
96:	learn: 5.0303964	test: 5.0290826	best: 5.0290826 (96)	total: 6.53s	remaining: 27.1s
97:	learn: 5.0260638	test: 5.0247064	best: 5.0247064 (97)	total: 6.58s	remaining: 27s
98:	learn: 5.0181595	test: 5.0169437	best: 5.0169437 (98)	total: 6.64s	remaining: 26.9s
99:	learn: 5.0104232	test: 5.0094973	best: 5.0094973 (99)	total: 6.69s	remaining: 26.8s
100:	learn: 5.0052996	test: 5.0039306	best: 5.0039306 (100)	total: 6.75s	remaining: 26.7s
101:	learn: 4.9977510	test: 4.9955474	best: 4.9955474 (101)	total: 6.81s	remaining: 26.6s
102:	learn: 4.9925592	test: 4.9908327	best: 4.9908327 (102)	total: 6.94s	remaining: 26.7s
103:	learn: 4.9864878	test: 4.9850744	best: 4.9850744 (103)	total: 7.03s	remaining: 26.8s
104:	learn: 4.9798951	test: 4.9768949	best: 4.9768949 (104)	total: 7.13s	remaining: 26.8s
105:	learn: 4.9739506	tes

187:	learn: 4.7577267	test: 4.7586428	best: 4.7586428 (187)	total: 11.9s	remaining: 19.7s
188:	learn: 4.7568053	test: 4.7580040	best: 4.7580040 (188)	total: 11.9s	remaining: 19.6s
189:	learn: 4.7551305	test: 4.7559752	best: 4.7559752 (189)	total: 12s	remaining: 19.6s
190:	learn: 4.7518293	test: 4.7529421	best: 4.7529421 (190)	total: 12.1s	remaining: 19.5s
191:	learn: 4.7487105	test: 4.7497196	best: 4.7497196 (191)	total: 12.1s	remaining: 19.5s
192:	learn: 4.7470628	test: 4.7478124	best: 4.7478124 (192)	total: 12.2s	remaining: 19.4s
193:	learn: 4.7444529	test: 4.7452146	best: 4.7452146 (193)	total: 12.3s	remaining: 19.3s
194:	learn: 4.7429404	test: 4.7435492	best: 4.7435492 (194)	total: 12.3s	remaining: 19.3s
195:	learn: 4.7417037	test: 4.7429102	best: 4.7429102 (195)	total: 12.4s	remaining: 19.2s
196:	learn: 4.7407307	test: 4.7421686	best: 4.7421686 (196)	total: 12.4s	remaining: 19.1s
197:	learn: 4.7396169	test: 4.7407056	best: 4.7407056 (197)	total: 12.5s	remaining: 19.1s
198:	learn: 

282:	learn: 4.5743376	test: 4.5895962	best: 4.5895962 (282)	total: 17.2s	remaining: 13.2s
283:	learn: 4.5736654	test: 4.5894838	best: 4.5894838 (283)	total: 17.2s	remaining: 13.1s
284:	learn: 4.5726751	test: 4.5894894	best: 4.5894838 (283)	total: 17.3s	remaining: 13.1s
285:	learn: 4.5708956	test: 4.5875775	best: 4.5875775 (285)	total: 17.4s	remaining: 13s
286:	learn: 4.5686642	test: 4.5855028	best: 4.5855028 (286)	total: 17.4s	remaining: 12.9s
287:	learn: 4.5681343	test: 4.5852672	best: 4.5852672 (287)	total: 17.5s	remaining: 12.8s
288:	learn: 4.5667063	test: 4.5834070	best: 4.5834070 (288)	total: 17.5s	remaining: 12.8s
289:	learn: 4.5642617	test: 4.5804671	best: 4.5804671 (289)	total: 17.6s	remaining: 12.7s
290:	learn: 4.5637329	test: 4.5801790	best: 4.5801790 (290)	total: 17.6s	remaining: 12.6s
291:	learn: 4.5627619	test: 4.5796109	best: 4.5796109 (291)	total: 17.7s	remaining: 12.6s
292:	learn: 4.5591645	test: 4.5754620	best: 4.5754620 (292)	total: 17.7s	remaining: 12.5s
293:	learn: 

377:	learn: 4.4402089	test: 4.4758169	best: 4.4758169 (377)	total: 22.2s	remaining: 7.17s
378:	learn: 4.4392956	test: 4.4752140	best: 4.4752140 (378)	total: 22.3s	remaining: 7.11s
379:	learn: 4.4385096	test: 4.4748419	best: 4.4748419 (379)	total: 22.3s	remaining: 7.05s
380:	learn: 4.4378045	test: 4.4741674	best: 4.4741674 (380)	total: 22.4s	remaining: 6.99s
381:	learn: 4.4364576	test: 4.4730429	best: 4.4730429 (381)	total: 22.4s	remaining: 6.92s
382:	learn: 4.4354698	test: 4.4725375	best: 4.4725375 (382)	total: 22.5s	remaining: 6.87s
383:	learn: 4.4335704	test: 4.4701822	best: 4.4701822 (383)	total: 22.5s	remaining: 6.81s
384:	learn: 4.4313697	test: 4.4681173	best: 4.4681173 (384)	total: 22.6s	remaining: 6.75s
385:	learn: 4.4307215	test: 4.4679840	best: 4.4679840 (385)	total: 22.6s	remaining: 6.69s
386:	learn: 4.4291022	test: 4.4657548	best: 4.4657548 (386)	total: 22.7s	remaining: 6.63s
387:	learn: 4.4268591	test: 4.4637907	best: 4.4637907 (387)	total: 22.7s	remaining: 6.57s
388:	learn

...

200:	learn: 4.0414215	test: 4.3033856	best: 4.3033856 (200)	total: 6m 34s	remaining: 9m 46s
201:	learn: 4.0353447	test: 4.2963993	best: 4.2963993 (201)	total: 6m 36s	remaining: 9m 44s
202:	learn: 4.0326029	test: 4.2951465	best: 4.2951465 (202)	total: 6m 38s	remaining: 9m 42s
203:	learn: 4.0314183	test: 4.2942397	best: 4.2942397 (203)	total: 6m 38s	remaining: 9m 37s
204:	learn: 4.0270303	test: 4.2921214	best: 4.2921214 (204)	total: 6m 39s	remaining: 9m 35s
205:	learn: 4.0255373	test: 4.2922659	best: 4.2921214 (204)	total: 6m 41s	remaining: 9m 32s
206:	learn: 4.0236133	test: 4.2923191	best: 4.2921214 (204)	total: 6m 43s	remaining: 9m 31s
207:	learn: 4.0215120	test: 4.2925215	best: 4.2921214 (204)	total: 6m 45s	remaining: 9m 29s
208:	learn: 4.0174906	test: 4.2894302	best: 4.2894302 (208)	total: 6m 47s	remaining: 9m 27s
209:	learn: 4.0119220	test: 4.2846069	best: 4.2846069 (209)	total: 6m 50s	remaining: 9m 26s
210:	learn: 4.0079457	test: 4.2813969	best: 4.2813969 (210)	total: 6m 51s	remain

290:	learn: 3.7475701	test: 4.1461497	best: 4.1461497 (290)	total: 8m 57s	remaining: 6m 25s
291:	learn: 3.7439220	test: 4.1431693	best: 4.1431693 (291)	total: 8m 58s	remaining: 6m 23s
292:	learn: 3.7417407	test: 4.1429711	best: 4.1429711 (292)	total: 9m	remaining: 6m 21s
293:	learn: 3.7390079	test: 4.1427824	best: 4.1427824 (293)	total: 9m 2s	remaining: 6m 19s
294:	learn: 3.7368073	test: 4.1426430	best: 4.1426430 (294)	total: 9m 3s	remaining: 6m 17s
295:	learn: 3.7330240	test: 4.1394734	best: 4.1394734 (295)	total: 9m 5s	remaining: 6m 15s
296:	learn: 3.7303460	test: 4.1394692	best: 4.1394692 (296)	total: 9m 6s	remaining: 6m 13s
297:	learn: 3.7253946	test: 4.1356237	best: 4.1356237 (297)	total: 9m 8s	remaining: 6m 11s
298:	learn: 3.7231862	test: 4.1357972	best: 4.1356237 (297)	total: 9m 9s	remaining: 6m 9s
299:	learn: 3.7202268	test: 4.1360543	best: 4.1356237 (297)	total: 9m 11s	remaining: 6m 7s
300:	learn: 3.7178362	test: 4.1352157	best: 4.1352157 (300)	total: 9m 12s	remaining: 6m 5s
3

379:	learn: 3.5391454	test: 4.0956827	best: 4.0955222 (378)	total: 11m 15s	remaining: 3m 33s
380:	learn: 3.5373639	test: 4.0956547	best: 4.0955222 (378)	total: 11m 17s	remaining: 3m 31s
381:	learn: 3.5335152	test: 4.0933896	best: 4.0933896 (381)	total: 11m 18s	remaining: 3m 29s
382:	learn: 3.5318843	test: 4.0935155	best: 4.0933896 (381)	total: 11m 20s	remaining: 3m 27s
383:	learn: 3.5296870	test: 4.0934881	best: 4.0933896 (381)	total: 11m 21s	remaining: 3m 25s
384:	learn: 3.5262246	test: 4.0900775	best: 4.0900775 (384)	total: 11m 23s	remaining: 3m 24s
385:	learn: 3.5237643	test: 4.0903102	best: 4.0900775 (384)	total: 11m 24s	remaining: 3m 22s
386:	learn: 3.5216361	test: 4.0905759	best: 4.0900775 (384)	total: 11m 26s	remaining: 3m 20s
387:	learn: 3.5199967	test: 4.0905283	best: 4.0900775 (384)	total: 11m 27s	remaining: 3m 18s
388:	learn: 3.5175276	test: 4.0904312	best: 4.0900775 (384)	total: 11m 29s	remaining: 3m 16s
389:	learn: 3.5141916	test: 4.0895966	best: 4.0895966 (389)	total: 11m

468:	learn: 3.3501934	test: 4.0681188	best: 4.0680571 (467)	total: 13m 37s	remaining: 54s
469:	learn: 3.3474997	test: 4.0680376	best: 4.0680376 (469)	total: 13m 39s	remaining: 52.3s
470:	learn: 3.3456376	test: 4.0678801	best: 4.0678801 (470)	total: 13m 40s	remaining: 50.5s
471:	learn: 3.3437056	test: 4.0676391	best: 4.0676391 (471)	total: 13m 42s	remaining: 48.8s
472:	learn: 3.3403726	test: 4.0676102	best: 4.0676102 (472)	total: 13m 43s	remaining: 47s
473:	learn: 3.3385221	test: 4.0673414	best: 4.0673414 (473)	total: 13m 45s	remaining: 45.3s
474:	learn: 3.3358560	test: 4.0670996	best: 4.0670996 (474)	total: 13m 46s	remaining: 43.5s
475:	learn: 3.3338587	test: 4.0671103	best: 4.0670996 (474)	total: 13m 48s	remaining: 41.8s
476:	learn: 3.3315078	test: 4.0661562	best: 4.0661562 (476)	total: 13m 49s	remaining: 40s
477:	learn: 3.3292340	test: 4.0659662	best: 4.0659662 (477)	total: 13m 51s	remaining: 38.3s
478:	learn: 3.3268701	test: 4.0664077	best: 4.0659662 (477)	total: 13m 52s	remaining: 

KeyboardInterrupt: 

In [182]:
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score
# predv = model.predict(te)
# rmse = (np.sqrt(mean_squared_error(tey, predv)))
# r2 = r2_score(tey, predv)
# print('RMSE: {:.2f}'.format(rmse))
# print('R2: {:.2f}'.format(r2))
# pred = polyreg.predict(te_data)

In [249]:
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score
# predv = model.predict(te)
# rmse = (np.sqrt(mean_squared_error(tey, predv)))
# r2 = r2_score(tey, predv)
# print('RMSE: {:.2f}'.format(rmse))
# print('R2: {:.2f}'.format(r2))
predt = model.predict(te_data)

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.

In [247]:
sub=pd.DataFrame()
sub['ID']= id_
sub['Time_taken (min)'] = predt
sub.to_csv(r"C:\Users\admin\Downloads\dataset\hellosublgbm_wh.csv",index=False)